The goal of this notebook is to gather, analyze and visualize the clubs that the majority of FIFA World Cup players play for. I will gather the data using Wikipedia ("xxxx_FIFA_World_Cup_squads") and use the wikipedia library to do so. I will gather multiple years of data so that we can determine **the strongest soccer clubs in the world**. This is important b/c it's not clear which leagues/clubs are the most dominant for a long period of time.

# Imports

In [1]:
import pandas as pd
import wikitables
from wikitables import import_tables
import jenkspy

## Functions

In [2]:
def fix_country_names(df):
    new_list = []
    for item in df['Country']:
        if str(item) ==  'FRA 1974 France':
            new_list.append('France')
        elif str(item[0:2]).isupper():
            new_list.append(item[4:])
        else:
            new_list.append(item)
    return new_list
def make_df_fields_numeric(df):
    df['Players'] = df['Players'].astype(str, errors='ignore')
    df['Players'] = pd.to_numeric(df['Players'], errors='coerce')
    df['Outside national squad'] = df['Outside national squad'].astype(str, errors='ignore')
    df['Outside national squad'] = pd.to_numeric(df['Outside national squad'], errors='coerce')
    df['Lower tier players'] = df['Lower tier players'].astype(str, errors='ignore')
    df['Lower tier players'] = pd.to_numeric(df['Lower tier players'], errors='coerce')
    return df
def make_cols_strings(df, columns=[]):
    for item in columns:
        df[item] = df[item].astype(str, errors='ignore')
    return df

## Data Import using wikitables

In [3]:
# importing a 2022 table seperately b/c it wasn't loaded with wikitables
countries_df_2022 = pd.read_csv("./data/2022_FIFA_World_Cup_squads_33.csv")

In [4]:
# there are summary tables at the bottom of each page
## I'll just import all of those into a single table 
tables_2022 = import_tables('2022 FIFA World Cup squads')
tables_2018 = import_tables('2018 FIFA World Cup squads')
tables_2014 = import_tables('2014 FIFA World Cup squads')
tables_2010 = import_tables('2010 FIFA World Cup squads')
tables_2006 = import_tables('2006 FIFA World Cup squads') 
# the following 2 only have 1 table
tables_2002 = import_tables('2002 FIFA World Cup squads')
tables_1998 = import_tables('1998 FIFA World Cup squads')

In [5]:
all_tables = [tables_2022, tables_2018, tables_2014, tables_2010, tables_2006, tables_2002, tables_1998]

# Performing Data Checks on the tables

In [6]:
# how many tables are in each page's list of tables?
for table in all_tables:
    print(f"{len(table)}")

5
4
3
4
2
1
1


In [7]:
# finding out what info is in each table
for table in tables_2022:
    print(table)
    print(table.rows[0].keys())
    print(sep = 'n')

<WikiTable '2022 FIFA World Cup squads[0]'>
dict_keys(['Players', 'Clubs'])

<WikiTable '2022 FIFA World Cup squads[1]'>
dict_keys(['Players', 'Clubs'])

<WikiTable '2022 FIFA World Cup squads[2]'>
dict_keys(['Confederation', 'Players', 'Percentage'])

<WikiTable '2022 FIFA World Cup squads[3]'>
dict_keys(['Average age', 'Countries'])

<WikiTable '2022 FIFA World Cup squads[4]'>
dict_keys(['Number', 'Country', 'Coaches'])



The type of info in the tables that I care about is **club, club country, and league system**

In [8]:
# removing unnecessary tables 
tables_2022 = tables_2022[0:3]
tables_2018 = tables_2018[0:3]
tables_2014 = tables_2014[0:2]
tables_2010 = tables_2010[0:2]
# re-assigning them to the main table
all_tables = [tables_2022, tables_2018, tables_2014, tables_2010, tables_2006, tables_2002, tables_1998]

In [9]:
# unique sets of keys in a table
unique_keys = []
for page in all_tables:
    for table in page:
        if table.rows[0].keys() not in unique_keys:
            unique_keys.append(table.rows[0].keys())

In [10]:
# country is a column in all of the country tables
unique_keys

[dict_keys(['Players', 'Clubs']),
 dict_keys(['Confederation', 'Players', 'Percentage']),
 dict_keys(['Country', 'Players', 'Percentage', 'Outsidenationalsquad', 'Lower tier  players']),
 dict_keys(['Club', 'Players']),
 dict_keys(['Confederation', 'Players']),
 dict_keys(['Country', 'Players', 'Outsidenationalsquad']),
 dict_keys(['Players', 'England', 'Spain', 'Italy', 'Germany', 'France', 'Netherlands', 'Other UEFA', 'Other regions']),
 dict_keys(['Country', 'Players', 'Percent', 'Outside  national  squad']),
 dict_keys(['Country', 'Players', 'Percent', 'Outside national squad'])]

# Turning Tables into DataFrames 

## League Country

In [11]:
# identifying the country tables
country_tables = []
for page in all_tables:
    for table in page:
        if "Country" in table.rows[0].keys():
            country_tables.append(table)

In [12]:
for name in country_tables:
    print(f"{name}")
    print(name.rows[0].keys())
    print(sep = 'n')

<WikiTable '2018 FIFA World Cup squads[0]'>
dict_keys(['Country', 'Players', 'Percentage', 'Outsidenationalsquad', 'Lower tier  players'])

<WikiTable '2014 FIFA World Cup squads[0]'>
dict_keys(['Country', 'Players', 'Outsidenationalsquad'])

<WikiTable '2010 FIFA World Cup squads[1]'>
dict_keys(['Country', 'Players', 'Percent', 'Outside  national  squad'])

<WikiTable '2006 FIFA World Cup squads[0]'>
dict_keys(['Country', 'Players', 'Percent', 'Outside  national  squad'])

<WikiTable '2002 FIFA World Cup squads[0]'>
dict_keys(['Country', 'Players', 'Percent', 'Outside  national  squad'])

<WikiTable '1998 FIFA World Cup squads[0]'>
dict_keys(['Country', 'Players', 'Percent', 'Outside national squad'])



## Turning them into seperate dataframes

In [13]:
countries_df_2018 = pd.DataFrame.from_dict(country_tables[0].rows)
countries_df_2014 = pd.DataFrame.from_dict(country_tables[1].rows)
countries_df_2010 = pd.DataFrame.from_dict(country_tables[2].rows)
countries_df_2006 = pd.DataFrame.from_dict(country_tables[3].rows)
countries_df_2002 = pd.DataFrame.from_dict(country_tables[4].rows)
countries_df_1998 = pd.DataFrame.from_dict(country_tables[5].rows)

In [14]:
countries_df_list = [countries_df_2022, countries_df_2018, countries_df_2014, 
                    countries_df_2010, countries_df_2006, countries_df_2002,
                    countries_df_1998]

In [15]:
# changing the tables to have the correct column names
column_name_replacements = {"Percentage": "Percent",
                           "Outsidenationalsquad": 'Outside national squad',
                           'Outside  national  squad': 'Outside national squad',
                           'Outside\nnational\nsquad': 'Outside national squad',
                           'Lower tier \n players': 'Lower tier players',
                           'Lower tier  players': 'Lower tier players'}

In [16]:
for df in countries_df_list:
    df.rename(columns=column_name_replacements,
              inplace=True)
    if 'Percent' in df.columns:
        df.drop(columns='Percent', inplace=True)

In [17]:
# what column names are still wonky
# all_country_column_names = []
# for df in countries_df_list:
#     for col_name in df.columns:
#         if col_name not in all_country_column_names:
#             all_country_column_names.append(col_name)

## Combining Country dataframes

In [18]:
# adding year to each df
countries_df_2022['Year']= '2022'
countries_df_2018['Year']= '2018'
countries_df_2014['Year']= '2014'
countries_df_2010['Year']= '2010'
countries_df_2006['Year']= '2006'
countries_df_2002['Year']= '2002'  
countries_df_1998['Year']= '1998'

In [19]:
# adding a blank column if it's missing percent or lower_tier_national_players
countries_df_2014['Lower tier players'] = 0
countries_df_2010['Lower tier players'] = 0
countries_df_2006['Lower tier players'] = 0
countries_df_2002['Lower tier players'] = 0
countries_df_1998['Lower tier players'] = 0

In [20]:
countries_df_2010['Country'] = countries_df_2010['Country'].astype(str)
countries_df_2014['Country'] = countries_df_2014['Country'].astype(str)
countries_df_2018['Country'] = countries_df_2018['Country'].astype(str)
new_2010_country_list = fix_country_names(countries_df_2010)
new_2014_country_list = fix_country_names(countries_df_2014)
new_2018_country_list = fix_country_names(countries_df_2018)

In [21]:
countries_df_2010['Country'] = new_2010_country_list
countries_df_2014['Country'] = new_2014_country_list
countries_df_2018['Country'] = new_2018_country_list

In [22]:
countries_df_2018 = make_df_fields_numeric(countries_df_2018)
countries_df_2014 = make_df_fields_numeric(countries_df_2014)
countries_df_2010 = make_df_fields_numeric(countries_df_2010)
countries_df_2006 = make_df_fields_numeric(countries_df_2006)
countries_df_2002 = make_df_fields_numeric(countries_df_2002)
countries_df_1998 = make_df_fields_numeric(countries_df_1998)

In [23]:
countries_df_combined_original = pd.concat(countries_df_list).reset_index(drop=True)
countries_df_combined = pd.concat(countries_df_list).reset_index(drop=True)

### Data Cleaning

In [24]:
countries_df_combined['Country'] = countries_df_combined['Country'].astype(str)

In [25]:
# removing total rows
bad_values_list = ['Total', 'Others']
countries_df_combined = countries_df_combined[countries_df_combined['Country'].isin(bad_values_list) == False]
countries_df_combined_original = countries_df_combined_original[countries_df_combined_original['Country'].isin(['Total']) == False]

In [26]:
countries_df_combined = make_df_fields_numeric(countries_df_combined)
countries_df_combined_original = make_df_fields_numeric(countries_df_combined_original)

In [27]:
# countries_df_combined['Lower tier players'] = pd.to_numeric(countries_df_combined['Lower tier players']
#                                                             , errors='coerce')
# countries_df_combined['Players'] = pd.to_numeric(countries_df_combined['Players']
#                                                             , errors='coerce')
# countries_df_combined['Outside national squad'] = pd.to_numeric(countries_df_combined['Outside national squad']
#                                                             , errors='coerce')

In [28]:
# most tables only show the top 10 countries so it's not fair to compare the less dominant countries
countries_df_groupby = countries_df_combined.groupby('Country').sum()

In [71]:
country_groupby_percentages_of_total = [round((x*100)/countries_df_combined_original.Players.sum(), 2)
                                        for x in countries_df_groupby['Players'].values]
country_w_percentage_of_total = list(zip(countries_df_groupby.index, country_groupby_percentages_of_total))

In [72]:
country_w_percentage_of_total.sort(key=lambda x:x[1], reverse=True)
country_w_percentage_of_total[0:8]

[('England', 11.18),
 ('Germany', 6.85),
 ('Italy', 6.81),
 ('Spain', 6.52),
 ('France', 4.71),
 ('Mexico', 2.31),
 ('Saudi Arabia', 1.87),
 ('Netherlands', 1.67)]

In [73]:
# raw total
countries_df_groupby.sort_values(by='Players', ascending=False)[0:8]

,Players,Outside national squad,Lower tier players
Country,,,
England,808,648.0,54
Germany,495,362.0,8
Italy,492,384.0,3
Spain,471,341.0,5
France,340,279.0,3
Mexico,167,54.0,1
Saudi Arabia,135,21.0,2
Netherlands,121,66.0,0


In [74]:
# percentage of ALL players
print(countries_df_combined_original.Players.sum())
# there are about 1000 players in the world cup 
## seems a pretty close since 32 teams * 32 players 
countries_df_combined_original.Players.sum()/7

7226


1032.2857142857142

# Club Data

## Finding the Tables

In [75]:
# club, players are the tables that are easiest to work with
## 2022, 2018, 2014, and 2006 have nice and neat tables
# identifying the country tables
club_tables = []
for page in all_tables:
    for table in page:
        if "Club" in table.rows[0].keys():
            club_tables.append(table)
        elif "Clubs" in table.rows[0].keys():
            club_tables.append(table)

In [76]:
# 2022 split the tables into more than 10 players and 10 or less players
for name in club_tables:
    print(f"{name}")
    print(name.rows[0].keys())
    print(sep = 'n')

<WikiTable '2022 FIFA World Cup squads[0]'>
dict_keys(['Players', 'Clubs'])

<WikiTable '2022 FIFA World Cup squads[1]'>
dict_keys(['Players', 'Clubs'])

<WikiTable '2018 FIFA World Cup squads[1]'>
dict_keys(['Club', 'Players'])

<WikiTable '2014 FIFA World Cup squads[1]'>
dict_keys(['Club', 'Players'])

<WikiTable '2006 FIFA World Cup squads[1]'>
dict_keys(['Players', 'Clubs'])



## Joining the Tables together

In [77]:
clubs_df_2022_1 = pd.DataFrame.from_dict(club_tables[0].rows)
clubs_df_2022_2 = pd.DataFrame.from_dict(club_tables[1].rows)
clubs_df_2018 = pd.DataFrame.from_dict(club_tables[2].rows)
clubs_df_2014 = pd.DataFrame.from_dict(club_tables[3].rows)
# clubs_df_2006 = pd.DataFrame.from_dict(club_tables[4].rows)

In [78]:
clubs_df_list = [clubs_df_2022_1, clubs_df_2022_2, clubs_df_2018, clubs_df_2014]
#, clubs_df_2006

In [79]:
for df in clubs_df_list:
    df.rename(columns={'Club': "Clubs"}, inplace=True)

In [80]:
new_col_order = ['Players', 'Clubs']
clubs_df_2018 = clubs_df_2018[new_col_order]
clubs_df_2014 = clubs_df_2014[new_col_order]

In [81]:
# adding the year
clubs_df_2022_1['Year'] = '2022'
clubs_df_2022_2['Year'] = '2022'
clubs_df_2018['Year'] = '2018'
clubs_df_2014['Year'] = '2014'
# clubs_df_2006['Year'] = '2006'

/Users/gregfeliu/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/gregfeliu/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [82]:
clubs_df_2022_1 = make_cols_strings(clubs_df_2022_1, columns=['Players', 'Clubs'])
clubs_df_2022_2 = make_cols_strings(clubs_df_2022_2, columns=['Players', 'Clubs'])
clubs_df_2018 = make_cols_strings(clubs_df_2018, columns=['Players', 'Clubs'])
clubs_df_2014 = make_cols_strings(clubs_df_2014, columns=['Players', 'Clubs'])
# clubs_df_2006 = make_cols_strings(clubs_df_2006, columns=['Players', 'Clubs'])

/Users/gregfeliu/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [83]:
clubs_df_2022_1['Players'] = pd.to_numeric(clubs_df_2022_1['Players'], errors='coerce')
clubs_df_2022_2['Players'] = pd.to_numeric(clubs_df_2022_2['Players'], errors='coerce')
clubs_df_2018['Players'] = pd.to_numeric(clubs_df_2018['Players'], errors='coerce')
clubs_df_2014['Players'] = pd.to_numeric(clubs_df_2014['Players'], errors='coerce')
# clubs_df_2006['Players'] = pd.to_numeric(clubs_df_2006['Players'], errors='coerce')

/Users/gregfeliu/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/gregfeliu/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [84]:
clubs_df_list = [clubs_df_2022_1, clubs_df_2022_2, clubs_df_2018, clubs_df_2014]
#  clubs_df_2006
clubs_df_combined_original = pd.concat(clubs_df_list).reset_index(drop=True)
clubs_df_combined = pd.concat(clubs_df_list).reset_index(drop=True)

In [85]:
# only rows with more than 10 players since that's the min cut off from 2018 and earlier
all_clubs = clubs_df_combined.copy()
clubs_df_combined = clubs_df_combined[clubs_df_combined['Players'] >= 10].reset_index(drop=True)

## Counting the number of times a club appears

In [86]:
# will need to make a dict that accounts for each team and adds the number of players
country_dict = {}
# will have a dictionary for each club and a count of each club after this 

In [87]:
# find all of the countries in the data set
for row in clubs_df_combined['Clubs']:
    split_clubs = row.split()
    for word in split_clubs:
        if word.isupper():
            if word not in country_dict.keys():
                country_dict[word] = None
#     if(words[i][0].isupper() and words[i+1][0].isupper()):

In [88]:
# find the index of the countries and then combine the values in between the countries
all_club_names = []
for row in clubs_df_combined['Clubs']:
   # find the index of the countries and then combine the values in between the countries
    country_indices = []
    split_clubs = row.split()
    if "1974" in split_clubs: # for the 1974 paris st. germain
        split_clubs.remove("1974")
    for idx, item in enumerate(split_clubs):
        # identify country and record idx 
        if item in country_dict.keys():
            country_indices.append(idx)
    country_indices.append(len(split_clubs))
    # find the club names between these indices
    club_name_list = []
    sub_club_name_list = []
    for x in range(len(split_clubs)+1):
        if x not in country_indices:
            sub_club_name_list.append(split_clubs[x])
        if x in country_indices and len(sub_club_name_list) != 0:
            club_name_list.append(" ".join(sub_club_name_list))
            sub_club_name_list = []
    all_club_names.append(club_name_list)

In [89]:
# getting all of the club names into a dict
club_name_appearances_dict = {}
for item in all_club_names:
    if len(item) == 1:
        club_name_appearances_dict[item[0]] = 0
    elif len(item) > 1:
        for thing in item:
            club_name_appearances_dict[thing] = 0

In [90]:
# finding the country of each club 
clubs_w_countries = {key: None for key in club_name_appearances_dict.keys()}
for row in clubs_df_combined['Clubs']:
    country_names = None
    split_row = row.split()
    for key in clubs_w_countries.keys():
        first_word = key.split()[0]
        if first_word in split_row:
            club_index = split_row.index(first_word)
            country = split_row[club_index - 1]
            clubs_w_countries[key] = country
# clubs_w_countries

In [91]:
# adding the number of players to each club
for idx, row in enumerate(range(len(clubs_df_combined))):
    for key, value in club_name_appearances_dict.items():
        if key in clubs_df_combined['Clubs'][idx]:
            club_name_appearances_dict[key] += clubs_df_combined['Players'][idx]

In [92]:
# The number of times that the club has sent more than 10 players
club_name_count_dict = {key: 0 for key in club_name_appearances_dict}
# adding the number of times the club appears
for idx, row in enumerate(range(len(clubs_df_combined))):
    for key, value in club_name_count_dict.items():
        if key in clubs_df_combined['Clubs'][idx]:
            club_name_count_dict[key] += 1

## Creating a Dataframe with all of the clubs

In [93]:
clubs_more_than_10_players_df = pd.DataFrame.from_dict(club_name_appearances_dict, orient='index')
clubs_more_than_10_players_df['Number_of_times_in_world_cup'] = club_name_count_dict.values()
clubs_more_than_10_players_df['Country'] = clubs_w_countries.values()

In [94]:
# changning from 1974
clubs_more_than_10_players_df.at['Paris Saint-Germain', 'Country'] = 'FRA'
clubs_more_than_10_players_df.columns = ["Players_sent", "Number_of_times_in_world_cup", "Country"]

In [95]:
clubs_more_than_10_players_df.sort_values(by=["Players_sent", "Number_of_times_in_world_cup"], ascending=False)

,Players_sent,Number_of_times_in_world_cup,Country
Barcelona,44,3,ESP
Manchester City,42,3,ENG
Bayern Munich,41,3,GER
Real Madrid,40,3,ESP
Manchester United,39,3,ENG
Chelsea,35,3,ENG
Juventus,34,3,ITA
Paris Saint-Germain,33,3,FRA
Tottenham Hotspur,23,2,ENG
Arsenal,20,2,ENG


## Natural Breaks

In [96]:
breaks = jenkspy.jenks_breaks(clubs_more_than_10_players_df['Players_sent'], nb_class=2)
print(breaks)

[10.0, 23.0, 44.0]


In [97]:
clubs_more_than_10_players_df['cut_jenks'] = pd.cut(clubs_more_than_10_players_df['Players_sent'],
                        bins=breaks,
                        labels=['Star clubs', 'Superstar clubs',],
                        include_lowest=True)
clubs_more_than_10_players_df

,Players_sent,Number_of_times_in_world_cup,Country,cut_jenks
Barcelona,44,3,ESP,Superstar clubs
Bayern Munich,41,3,GER,Superstar clubs
Manchester City,42,3,ENG,Superstar clubs
Al-Sadd,15,1,QAT,Star clubs
Manchester United,39,3,ENG,Superstar clubs
Real Madrid,40,3,ESP,Superstar clubs
Al Hilal,12,1,KSA,Star clubs
Atlético Madrid,12,1,ESP,Star clubs
Chelsea,35,3,ENG,Superstar clubs
Ajax,11,1,NED,Star clubs


## Grouping by Country

In [98]:
clubs_by_country_count_appearences = clubs_more_than_10_players_df.groupby('Country').count()
clubs_by_country_count_appearences

,Players_sent,Number_of_times_in_world_cup,cut_jenks
Country,,,
ENG,6,6,6
ESP,4,4,4
FRA,1,1,1
GER,2,2,2
ITA,2,2,2
KSA,1,1,1
NED,1,1,1
QAT,1,1,1


# Exporting Data

In [99]:
# country tables
countries_df_combined.to_csv("./data/countries_df_combined.csv")
countries_df_groupby.to_csv("./data/countries_df_groupby.csv")

In [100]:
# clubs tabls 
clubs_more_than_10_players_df.to_csv("./data/clubs_more_than_10_players_df.csv")
clubs_by_country_count_appearences.to_csv("./data/clubs_by_country_count_appearences.csv")
all_clubs.to_csv("./data/all_clubs.csv")

# Links

## Wikipedia Tables

[2022](https://en.wikipedia.org/wiki/2022_FIFA_World_Cup_squads)
[2018](https://en.wikipedia.org/wiki/2018_FIFA_World_Cup_squads)
[2010](https://en.wikipedia.org/wiki/2010_FIFA_World_Cup_squads)
[2006](https://en.wikipedia.org/wiki/2006_FIFA_World_Cup_squads)
[2002](https://en.wikipedia.org/wiki/2002_FIFA_World_Cup_squads)
[1998](https://en.wikipedia.org/wiki/1998_FIFA_World_Cup_squads)

## Best League Ranking

[The best 33](https://www.globalfootballrankings.com)
[Top 20](https://soccerprime.com/best-soccer-leagues/)
[Top 10](https://sportsbrief.com/facts/top-listicles/29327-which-soccer-leagues-world-a-ranked-list/)

## Best Club Ranking

[538](https://projects.fivethirtyeight.com/soccer-predictions/global-club-rankings/)
[statista](https://www.statista.com/statistics/808025/best-soccer-club-teams-worldwide/)
[bleacher report](https://bleacherreport.com/articles/546420-the-100-greatest-soccer-clubs-in-the-world/)